# 6 Cnn Lstm Model

In [2]:
# cnn_model.py  (start in notebook cell, then export)

import sys
sys.path.append('./the .py files')  # relative path from notebook to .py directory



In [15]:
# cnn_lstm_model.py

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    Input,
    Conv1D,
    BatchNormalization,
    Dropout,
    MaxPooling1D,
    LSTM,
    Dense
)
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
from pre_processing import load_preprocessed_data
from evaluator import evaluate_model


def build_cnn_lstm_model(input_shape):
    """
    Build and compile a CNN→LSTM hybrid for RUL prediction.

    Args:
        input_shape (tuple): (timesteps, features), e.g. (30, 15).

    Returns:
        tf.keras.Model: compiled model ready for training.
    """
    inputs = Input(shape=input_shape)

    # Conv feature extraction
    x = Conv1D(64, kernel_size=5, padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = MaxPooling1D(pool_size=2)(x)

    x = Conv1D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = MaxPooling1D(pool_size=2)(x)

    # LSTM temporal modeling
    x = LSTM(64, return_sequences=False)(x)
    x = Dropout(0.2)(x)

    # Dense regression head
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    outputs = Dense(1, activation='linear')(x)

    model = Model(inputs=inputs, outputs=outputs, name='CNN_LSTM_Hybrid')
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss='mse',
        metrics=['mae']
    )
    return model


def train_cnn_lstm_model(
    model,
    X_train,
    y_train,
    X_val,
    y_val,
    epochs=40,
    batch_size=64,
    patience=4,
    verbose=1
):
    """
    Train the CNN–LSTM hybrid with early stopping.

    Args:
        model (tf.keras.Model): compiled model.
        X_train (np.ndarray): train inputs.
        y_train (np.ndarray): train targets.
        X_val, y_val: validation splits.
        epochs (int): max epochs.
        batch_size (int): batch size.
        patience (int): early-stopping patience on val_loss.
        verbose (int): verbosity level.

    Returns:
        tf.keras.callbacks.History: training history.
    """
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=patience,
            restore_best_weights=True
        )
    ]
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=verbose
    )
    return history


def predict_cnn_lstm_model(model, X):
    """
    Run inference and return 1D RUL predictions.

    Args:
        model (tf.keras.Model): trained model.
        X (np.ndarray): input windows.

    Returns:
        np.ndarray: shape (n_samples,) predicted RUL.
    """
    preds = model.predict(X, verbose=0)
    return preds.flatten()


def save_cnn_lstm_model(model, filename):
    """
    Save the hybrid model to disk.

    Args:
        model (tf.keras.Model): model to save.
        filename (str): path, e.g. 'cnn_lstm.fd001.keras' or '.h5'.
    """
    model.save(filename)


def load_cnn_lstm_model(filename):
    """
    Load a previously saved hybrid model.

    Args:
        filename (str): path where model was saved.

    Returns:
        tf.keras.Model: reconstructed model.
    """
    return load_model(filename)

def run_cnn_lstm_pipeline(
    npz_path="fd001_last.npz",
    epochs=40,
    batch_size=64,
    patience=4,
    verbose=1,
    model_name="CNN-LSTM"
):
    """
    Full CNN→LSTM pipeline:
      1. Load preprocessed data (.npz)
      2. Build & train the hybrid model
      3. Evaluate on validation set
      4. Attempt evaluate on test set (safely)

    Returns:
        model: the trained tf.keras.Model
        val_metrics: dict of RMSE & MAE on validation
        test_metrics: dict of RMSE & MAE on test (or {} if skipped)
    """
    # 1) Load data
    X_train, y_train, X_val, y_val, X_test, y_test = load_preprocessed_data(npz_path)

    # 2) Build & compile
    model = build_cnn_lstm_model(input_shape=X_train.shape[1:])

    # 3) Train
    train_cnn_lstm_model(
        model,
        X_train, y_train,
        X_val, y_val,
        epochs=epochs,
        batch_size=batch_size,
        patience=patience,
        verbose=verbose
    )

    # 4) Validation evaluation
    y_val_pred = predict_cnn_lstm_model(model, X_val)
    val_metrics = evaluate_model(y_val, y_val_pred, model_name + " (val)")

    # 5) Test evaluation (wrapped to never break)
    test_metrics = {}
    try:
        if (
            isinstance(X_test, np.ndarray) and isinstance(y_test, np.ndarray)
            and X_test.size > 0 and y_test.size > 0
        ):
            y_test_pred = predict_cnn_lstm_model(model, X_test)
            test_metrics = evaluate_model(y_test, y_test_pred, model_name + " (test)")
        else:
            print("⚠️ No valid test set found – skipping test evaluation.")
    except Exception as e:
        print(f"⚠️ Skipped test evaluation due to error: {e}")

    return model, val_metrics, test_metrics


In [8]:
# Cell: Test build_cnn_lstm_model

model = build_cnn_lstm_model(input_shape=(30, 15))
model.summary()


Model: "CNN_LSTM_Hybrid"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 30, 15)]          0         
                                                                 
 conv1d_4 (Conv1D)           (None, 30, 64)            4864      
                                                                 
 batch_normalization_2 (Batc  (None, 30, 64)           256       
 hNormalization)                                                 
                                                                 
 dropout_6 (Dropout)         (None, 30, 64)            0         
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 15, 64)           0         
 1D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 15, 128)       

In [9]:
# Cell: Test train_cnn_lstm_model on a small sample

# Load a small subset of data
from pre_processing import load_preprocessed_data

X_train, y_train, X_val, y_val, _, _ = load_preprocessed_data("fd001_last.npz")

# (Optional) Slice a tiny subset to speed up test
X_train_small, y_train_small = X_train[:512], y_train[:512]
X_val_small, y_val_small = X_val[:128], y_val[:128]

# Build model again just in case (fresh copy)
model = build_cnn_lstm_model(input_shape=(30, 15))

# Train on small subset
history = train_cnn_lstm_model(
    model,
    X_train_small, y_train_small,
    X_val_small, y_val_small,
    epochs=5,
    batch_size=64,
    patience=2,
    verbose=1
)


Epoch 1/5
8/8 [==============================] - 2s 57ms/step - loss: 8268.4082 - mae: 79.7793 - val_loss: 7869.9004 - val_mae: 78.2525
Epoch 2/5
8/8 [==============================] - 0s 12ms/step - loss: 7913.6401 - mae: 77.9237 - val_loss: 7608.7764 - val_mae: 76.8456
Epoch 3/5
8/8 [==============================] - 0s 12ms/step - loss: 7481.2256 - mae: 75.5446 - val_loss: 7259.4087 - val_mae: 74.8999
Epoch 4/5
8/8 [==============================] - 0s 12ms/step - loss: 6977.8447 - mae: 72.5691 - val_loss: 6748.6270 - val_mae: 71.9416
Epoch 5/5
8/8 [==============================] - 0s 12ms/step - loss: 6475.3628 - mae: 69.0237 - val_loss: 6097.1221 - val_mae: 67.6454


In [10]:
# Cell: Test predict_cnn_lstm_model

# Make predictions on validation set
y_val_pred = predict_cnn_lstm_model(model, X_val_small)

# Check output shape and some predictions
print("✅ Prediction shape:", y_val_pred.shape)
print("🔍 First 5 predictions:", y_val_pred[:5])

✅ Prediction shape: (128,)
🔍 First 5 predictions: [ 6.88867  14.758376 14.843136 14.56177  14.938363]


In [11]:
# Cell: Test save_cnn_lstm_model and load_cnn_lstm_model

# Save current model to disk
save_cnn_lstm_model(model, "test_cnn_lstm_model.keras")
print("✅ Model saved as 'test_cnn_lstm_model.keras'")

# Load it back
loaded_model = load_cnn_lstm_model("test_cnn_lstm_model.keras")
print("✅ Model loaded successfully")

# Re-run predictions using loaded model (sanity check)
y_val_pred_loaded = predict_cnn_lstm_model(loaded_model, X_val_small)
print("🔁 Predictions from loaded model (first 5):", y_val_pred_loaded[:5])


✅ Model saved as 'test_cnn_lstm_model.keras'
✅ Model loaded successfully
🔁 Predictions from loaded model (first 5): [ 6.88867  14.758376 14.843136 14.56177  14.938363]


In [17]:
# Cell: Test full run_cnn_lstm_pipeline

model, val_metrics, test_metrics = run_cnn_lstm_pipeline(
    npz_path="fd001_last.npz",
    epochs=20,
    batch_size=64,
    patience=3,
    verbose=1,
    model_name="CNN-LSTM Final"
)

Epoch 1/20
222/222 [==============================] - 4s 13ms/step - loss: 2971.0735 - mae: 41.4642 - val_loss: 571.5601 - val_mae: 19.9049
Epoch 2/20
222/222 [==============================] - 2s 10ms/step - loss: 526.0872 - mae: 17.7922 - val_loss: 361.0901 - val_mae: 15.4633
Epoch 3/20
222/222 [==============================] - 2s 10ms/step - loss: 418.5374 - mae: 15.6864 - val_loss: 225.6804 - val_mae: 11.8935
Epoch 4/20
222/222 [==============================] - 2s 11ms/step - loss: 344.5992 - mae: 14.2167 - val_loss: 272.6586 - val_mae: 13.4465
Epoch 5/20
222/222 [==============================] - 2s 11ms/step - loss: 292.6902 - mae: 13.1332 - val_loss: 163.6884 - val_mae: 10.4812
Epoch 6/20
222/222 [==============================] - 2s 11ms/step - loss: 274.2443 - mae: 12.6448 - val_loss: 129.2930 - val_mae: 9.4041
Epoch 7/20
222/222 [==============================] - 2s 11ms/step - loss: 241.5079 - mae: 11.9042 - val_loss: 109.9475 - val_mae: 8.6209
Epoch 8/20
222/222 [========

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).